In [5]:
import dagshub
import mlflow
from mlflow import MlflowClient

In [6]:
dagshub.init(repo_owner='zapatacc', repo_name='final-exam-pcd2024-autumn', mlflow=True)
mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
TRACKING_URI = mlflow.get_tracking_uri()

Accessing as Pepe-Chuy

Initialized MLflow to track repo "zapatacc/final-exam-pcd2024-autumn"

Repository zapatacc/final-exam-pcd2024-autumn initialized!

In [21]:
import mlflow

mlflow.set_experiment("jesus-carbajal-logreg-label-encoder")
client = MlflowClient()

# Get the experiment ID
experiment = client.get_experiment_by_name("jesus-carbajal-logreg-label-encoder")
experiment_id = experiment.experiment_id

#Get 100 experiments
runs = client.search_runs(
    experiment_ids=[experiment_id],
    run_view_type=mlflow.entities.ViewType.ACTIVE_ONLY,
    max_results=100,
    order_by=["start_time desc"]
)

best_run = 0
best_accuracy = 0

for run in runs:
    accuracy = run.data.metrics.get('accuracy', None)
    if accuracy is not None and accuracy > best_accuracy:
        best_accuracy = accuracy
        best_run = run

if best_run:
    print(f"Best Run ID: {best_run.info.run_id}")
    print(f"Accuracy: {best_accuracy}")


model_uri = f"runs:/{best_run.info.run_id}/model"
model_name = "jesus-carbajal-model"
registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version
client.set_registered_model_alias(
        name=model_name,
        alias="Champ",
        version=model_version
    )

Best Run ID: 8ccf9f0120494d78a04c99aa0b113d82
Accuracy: 0.5718428684418666


Successfully registered model 'jesus-carbajal-model'.
2024/11/21 02:05:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: jesus-carbajal-model, version 1
Created version '1' of model 'jesus-carbajal-model'.
/tmp/ipykernel_345485/302226163.py:36: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  model_version = client.get_latest_versions(name=model_name, stages=["None"])[0].version


In [24]:
run_uri = 'runs:/8ccf9f0120494d78a04c99aa0b113d82/pipeline_model'


In [25]:
dv = mlflow.pyfunc.load_model(run_uri)

In [27]:
dv.predict(["It was bad"])

array(['Credit card or prepaid card + General-purpose credit card or charge card'],
      dtype=object)